In [11]:
#! /usr/bin/env python

""" Phase-space Monte-Carlo event generator """

import numpy as np
import matplotlib.pyplot as plt
import phasespace as phsp

mtau = 1776.86
mpip = 139.57061
me = 0.5
alpha = 1 / 137

gev_to_mb = 0.389379  # 1 hbar^2 * c^2 * GeV^{-2} = 0.389379 mb
sigma0 = 4 * np.pi * alpha**2 / 3. * 10**12 * gev_to_mb

def velocity(s, m=mtau):
    """ """
    return np.sqrt(1-4*m**2/s)

def xsec(x, bias=0):
    """ Lower oeder QED calculation """
    f = np.zeros(x.shape)
    mask = x > 2*mtau + 0.395
    xi = (x - 2*mtau - 0.395)[mask]
    f[mask] = 0.12 + (0.1162*xi + (5.641*10**(-3))*xi**2) * xi**(-0.154)
    return f

def masses_3pi(mnu):
    """ Masses """
    return (mtau, [mpip, mpip, mpip, mnu])

def generate_tau_decay(mo, chs, n):
    """ returns (weights, particles) """
    return phsp.nbody_decay(mo, chs).generate(n_events=n)

def random_unit_vector():
    """ """
    pass

def boost_matrix(boost_vector):
    """ ___       Boost matrix                                ___
        |  g         -g*b*nx         -g*b*ny        -g*b*nz     |
        | -g*b*nx  1+(g-1)*nx^2      (g-1)*nx*ny    (g-1)*nx*nz |
        | -g*b*ny    (g-1)*ny*nx   1+(g-1)*ny^2     (g-1)*ny*nz |
        | -g*b*nz    (g-1)*nz*nx     (g-1)*nz*ny  1+(g-1)*nz^2  |
        ___                                                   ___
    """
    g = boost_vector[0]/mtau;
    b = np.sqrt(1 - 1/(g**2));
    p = np.sqrt(boost_vector[1]**2 + boost_vector[2]**2 + boost_vector[3]**2);
    nx, ny, nz = boost_vector[1]/p , boost_vector[2]/p, boost_vector[3]/p;
    return np.array([[g, -g*b*nx, -g*b*ny, -g*b*nz],[-g*b*nx, 1+(g-1)*nx**2, (g-1)*nx*ny, (g-1)*nx*nz],[-g*b*ny, (g-1)*ny*nx, 1+(g-1)*ny**2, (g-1)*ny*nz],[-g*b*nz, (g-1)*nz*nx, (g-1)*nz*ny, 1+(g-1)*nz**2]])

def boost(events, boosts):
    boosted = {key: np.zeros(shape=(events['p_0'].shape[0],4)) for key in events.keys()}
    for i in events.keys():
        newevents = np.array(events[i])[:,[3,0,1,2]]
        #boosted[i] = np.array([boost_matrix(boosts).dot(newevents[j]) for j in range(newevents.shape[0])]);
        boosted[i] = np.einsum('ijk,ik->ij', np.apply_along_axis(boost_matrix, -1, boosts), newevents)
    return boosted

def boost_to_lab_frame(events, sqrts, sigma):
    """ Events in tau frame to lab frame
        sqrts = sqrt(s), with the Mandelstam variable s
        sigma - Gaussian beam energy spread
    """
    N = events[1]['p_0'].shape[0]
    debeam = sigma*np.random.randn(N) + sqrts
    sgms = xsec(debeam)
    weights1 = np.array(events[0]*sgms);
    debeam = debeam[weights1 > 0]
    boosts = np.ones(shape = (debeam.shape[0], 4))
    boosts[:,0] = boosts[:,0]*debeam/2
    boosts[:,1] = boosts[:,1]*0
    boosts[:,2] = boosts[:,2]*0
    boosts[:,3] = (-1)*boosts[:,3]*np.sqrt((debeam**2)/4 - mtau**2)
    boosts = np.einsum('ijk,ik->ij', np.apply_along_axis(boost_matrix, -1, boosts1), boosts)
    newevents = {key: np.zeros(shape=(events[1]['p_0'].shape[0],4)) for key in events[1].keys()}
    for i in events[1].keys():
        newevents[i] = events[1][i][weights1 > 0]
    boosted = boost(newevents, boosts)
    return boosted
    

def main():
    """ Unit test """
    mtau, mchs = masses_3pi(0.)
    weights, particles = generate_tau_decay(mtau, mchs, 10**3)
    print(weights)
    print(particles)

if __name__ == '__main__':
    main()

tf.Tensor(
[0.06082589 0.04671022 0.06428706 0.03653647 0.06302272 0.09943803
 0.01243729 0.06659867 0.05736326 0.04554427 0.07179605 0.01947491
 0.07048717 0.09633231 0.0910876  0.07326277 0.06112863 0.05615962
 0.06681412 0.07830702 0.07184966 0.0976433  0.09541843 0.04755707
 0.09982463 0.09958515 0.02840096 0.01743786 0.06537278 0.09674371
 0.01673788 0.09743581 0.02703093 0.02574167 0.00104225 0.10085731
 0.04806083 0.02665442 0.07641497 0.01963012 0.07646588 0.08350697
 0.0590601  0.03839252 0.02200147 0.01671099 0.1001995  0.03466601
 0.05635409 0.05476648 0.03782818 0.06039926 0.02932214 0.0827014
 0.06603893 0.06895545 0.07066467 0.04743234 0.05436689 0.03579915
 0.09877324 0.07318115 0.06117546 0.04684529 0.0134325  0.09420813
 0.00529071 0.07584671 0.04148791 0.06919083 0.09371811 0.04962453
 0.01116769 0.09366373 0.00669765 0.03465766 0.04205215 0.07055336
 0.05355982 0.0120988  0.04732035 0.07391307 0.04131    0.06362525
 0.07779465 0.02637556 0.00715221 0.01248594 0.07048

In [12]:
mtau, mchs = masses_3pi(0.)
weights, particles = generate_tau_decay(mtau, mchs, 10**4)

In [14]:
a = np.array([weights, particles])
particles

{'p_0': <tf.Tensor: shape=(10000, 4), dtype=float64, numpy=
 array([[-225.35461827,  -16.69620404,  -53.4458454 ,  270.92412365],
        [-347.78993157, -650.85789388, -193.40800022,  775.53880899],
        [-284.74321551,  206.82916417,  -56.01830796,  382.72053503],
        ...,
        [ 360.24771491, -481.1732718 ,  182.14109538,  643.39837379],
        [ -21.12442467,  -30.90189295, -579.34765935,  597.09700541],
        [ 174.85238789,  -37.10640698,   65.31182433,  235.99964526]])>,
 'p_1': <tf.Tensor: shape=(10000, 4), dtype=float64, numpy=
 array([[ 225.28235617,  206.0193239 , -418.06217085,  536.14553592],
        [ -74.04767623,  248.36540207,  184.22383652,  347.25611358],
        [ 618.86222178,  165.15025047,  343.72039025,  740.1950531 ],
        ...,
        [-465.0718597 ,  274.35728219,  230.59481358,  603.50449559],
        [-138.50105649, -143.81187576,  465.30189643,  525.21444025],
        [ 500.33609358,   58.15236332,   58.51583795,  525.94863092]])>,
 'p_2': 

In [13]:
debeam = np.ones(shape = (10**4))*3600
boosts = np.ones(shape = (debeam.shape[0], 4))
boosts[:,0] = boosts[:,0]*debeam/2
boosts[:,1] = boosts[:,1]*0
boosts[:,2] = boosts[:,2]*0
boosts[:,3] = boosts[:,3]*np.sqrt((debeam**2)/4 - mtau**2)
particles1 = boost(particles, boosts)
particles1

{'p_0': array([[ 283.10589292, -225.35461827,  -16.69620404,  -98.00777633],
        [ 816.95373502, -347.78993157, -650.85789388, -321.49590273],
        [ 396.7747389 , -284.74321551,  206.82916417, -118.71493724],
        ...,
        [ 622.2864773 ,  360.24771491, -481.1732718 ,   80.3391005 ],
        [ 698.6764083 ,  -21.12442467,  -30.90189295, -683.56974774],
        [ 228.49828445,  174.85238789,  -37.10640698,   27.95116869]]),
 'p_1': array([[ 610.81708939,  225.28235617,  206.0193239 , -510.31505925],
        [ 321.95034331,  -74.04767623,  248.36540207,  130.39799493],
        [ 694.18208627,  618.86222178,  165.15025047,  228.3500798 ],
        ...,
        [ 574.02781656, -465.0718597 ,  274.35728219,  135.88313372],
        [ 456.71626372, -138.50105649, -143.81187576,  386.32291172],
        [ 523.32362071,  500.33609358,   58.15236332,  -25.87958505]]),
 'p_2': array([[ 288.58673829,   52.05739458,  172.40834366, -177.10940417],
        [ 497.21811833,  320.44820448, 

In [15]:
newevents = {key: np.zeros(shape=(particles1['p_0'].shape[0],4)) for key in particles1.keys()}
for i in particles1.keys():
    newevents[i] = np.array(particles1[i])[:,[1, 2, 3, 0]]
newevents

{'p_0': array([[-225.35461827,  -16.69620404,  -98.00777633,  283.10589292],
        [-347.78993157, -650.85789388, -321.49590273,  816.95373502],
        [-284.74321551,  206.82916417, -118.71493724,  396.7747389 ],
        ...,
        [ 360.24771491, -481.1732718 ,   80.3391005 ,  622.2864773 ],
        [ -21.12442467,  -30.90189295, -683.56974774,  698.6764083 ],
        [ 174.85238789,  -37.10640698,   27.95116869,  228.49828445]]),
 'p_1': array([[ 225.28235617,  206.0193239 , -510.31505925,  610.81708939],
        [ -74.04767623,  248.36540207,  130.39799493,  321.95034331],
        [ 618.86222178,  165.15025047,  228.3500798 ,  694.18208627],
        ...,
        [-465.0718597 ,  274.35728219,  135.88313372,  574.02781656],
        [-138.50105649, -143.81187576,  386.32291172,  456.71626372],
        [ 500.33609358,   58.15236332,  -25.87958505,  523.32362071]]),
 'p_2': array([[  52.05739458,  172.40834366, -177.10940417,  288.58673829],
        [ 320.44820448,  352.93709378, 

In [16]:
boosts1 = np.ones(shape = (debeam.shape[0], 4))
boosts1[:,0] = boosts[:,0]
boosts1[:,1] = boosts[:,1]
boosts1[:,2] = boosts[:,2]
boosts1[:,3] = (-1)*boosts[:,3]
boosted = boost(newevents, boosts1)
newevents1 = {key: np.zeros(shape=(particles1['p_0'].shape[0],4)) for key in particles1.keys()}
for i in particles1.keys():
    newevents1[i] = np.array(boosted[i])[:,[1, 2, 3, 0]]
newevents1

{'p_0': array([[-225.35461827,  -16.69620404,  -53.4458454 ,  270.92412365],
        [-347.78993157, -650.85789388, -193.40800022,  775.53880899],
        [-284.74321551,  206.82916417,  -56.01830796,  382.72053503],
        ...,
        [ 360.24771491, -481.1732718 ,  182.14109538,  643.39837379],
        [ -21.12442467,  -30.90189295, -579.34765935,  597.09700541],
        [ 174.85238789,  -37.10640698,   65.31182433,  235.99964526]]),
 'p_1': array([[ 225.28235617,  206.0193239 , -418.06217085,  536.14553592],
        [ -74.04767623,  248.36540207,  184.22383652,  347.25611358],
        [ 618.86222178,  165.15025047,  343.72039025,  740.1950531 ],
        ...,
        [-465.0718597 ,  274.35728219,  230.59481358,  603.50449559],
        [-138.50105649, -143.81187576,  465.30189643,  525.21444025],
        [ 500.33609358,   58.15236332,   58.51583795,  525.94863092]]),
 'p_2': array([[  52.05739458,  172.40834366, -132.69019616,  263.66883135],
        [ 320.44820448,  352.93709378, 

In [17]:
particles

{'p_0': <tf.Tensor: shape=(10000, 4), dtype=float64, numpy=
 array([[-225.35461827,  -16.69620404,  -53.4458454 ,  270.92412365],
        [-347.78993157, -650.85789388, -193.40800022,  775.53880899],
        [-284.74321551,  206.82916417,  -56.01830796,  382.72053503],
        ...,
        [ 360.24771491, -481.1732718 ,  182.14109538,  643.39837379],
        [ -21.12442467,  -30.90189295, -579.34765935,  597.09700541],
        [ 174.85238789,  -37.10640698,   65.31182433,  235.99964526]])>,
 'p_1': <tf.Tensor: shape=(10000, 4), dtype=float64, numpy=
 array([[ 225.28235617,  206.0193239 , -418.06217085,  536.14553592],
        [ -74.04767623,  248.36540207,  184.22383652,  347.25611358],
        [ 618.86222178,  165.15025047,  343.72039025,  740.1950531 ],
        ...,
        [-465.0718597 ,  274.35728219,  230.59481358,  603.50449559],
        [-138.50105649, -143.81187576,  465.30189643,  525.21444025],
        [ 500.33609358,   58.15236332,   58.51583795,  525.94863092]])>,
 'p_2': 